In [3]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [5]:
train_df = pd.read_pickle(".//train.pkl")
# test_df = pd.read_pickle("../input/hw2pkl/test.pkl")
train_df.head()

,ari_time,ari_title,price
0,-- 2010-02-05T11:54:40Z,-- Royal Opera House Chief Tony Hall Appointed...,0
1,-- 2006-12-14T15:12:30Z,-- Vale's Indonesia Nickel Unit May Cut Output...,0
2,-- 2008-12-29T20:53:34Z,-- 10 Mantras for Emerging Leaders in 2009,0
3,-- 2008-02-14T04:05:04Z,-- Hewlett-Packard Settles Claim by Reporters ...,0
4,-- 2008-02-14T21:14:32Z,-- New York's Dinallo Considers Splitting Bond...,0


In [9]:
corpus = pd.concat([train_df.ari_time, train_df.ari_title]).sample(frac=1)
corpus.head()

60502    -- Banca Popolare di Milano Says It Doesn't Ne...
34484    -- Goldman Sachs-Backed Emissions Exchange Is ...
36918    -- Barclays Hires Goldman's Koo, BNP's Lim to ...
52073    -- Apax, Guardian Studying $3.2 Billion Sale o...
29067    -- Mexican Tortillas Assure Carstens Lending R...
dtype: object

In [11]:
%%time
# CPU times: user 1min 11s
model = Word2Vec(corpus, size=250, iter=10)
model.save('word2vec.model')
# model = Word2Vec.load('word2vec.model')

CPU times: user 1min 11s, sys: 321 ms, total: 1min 11s
Wall time: 26.3 s
